In [ ]:
# Price Impact Demo

This notebook runs the price-impact example and saves the generated plot. It also demonstrates calling `MarketEngine.calculate_price_impact` for several sizes and displays the saved PNG.

# Install & Import Dependencies
# (Notebook-friendly checks; CI/runner likely has deps installed already)

import sys
import os
from pathlib import Path
import numpy as np
import matplotlib
matplotlib.use('Agg')  # safe for headless execution
import matplotlib.pyplot as plt

# quick version checks
print('Python', sys.version.splitlines()[0])
print('NumPy', np.__version__)

# Configure reproducibility and paths
ROOT = Path(os.getcwd())
ARTIFACTS = ROOT / 'artifacts'
ARTIFACTS.mkdir(exist_ok=True)

# Task 1: Generate price-impact curve (uses the project's script)
from scripts.price_impact_example import main as generate_example

png_path = ARTIFACTS / 'price_impact_example.png'
print('Generating price impact example plot ->', png_path)
generate_example(str(png_path))

# Inspect results
from IPython.display import Image, display
display(Image(str(png_path)))

# Quick table of impacts for a few sizes
from market_engine import MarketEngine
engine = MarketEngine(init_price=100.0, vol=0.0, rng=None)
mm = __import__('bots').MarketMaker(spread=1.0, size=1.0, jitter=0.0)
engine.register_bot(mm)
for _ in range(3):
    engine.step()

sizes = [1, 5, 10, 50, 100]
impacts = [(s, engine.calculate_price_impact('buy', s)) for s in sizes]
print('\nPrice impact (bps) by order size:')
for s, b in impacts:
    print(f'  size={s:3d} -> {b:.4f} bps')

# Save a small CSV summary
import csv
with open(ARTIFACTS / 'price_impact_summary.csv', 'w', newline='') as f:
    w = csv.writer(f)
    w.writerow(['size', 'impact_bps'])
    for s, b in impacts:
        w.writerow([s, b])

print('\nSaved CSV summary to', ARTIFACTS / 'price_impact_summary.csv')